# How much is your Fiat 500 worth?
## Skills: Python, Web scraping, Geopy, Regression

Download the Jupyter Notebook from [GitHub](https://nicomoriuala.github.io)

<a id='content' />

## Contents

1. [Introduction](#intro)
1. [Web Scraping](#scraping)
2. [Data Cleansing](#cleansing)
3. [Mapping](#mapping)
4. [A definir](#volatility)

<a id='intro' />

## Introduction
[(back to top)](#content)

In [1]:
#Importing packages
import pandas as pd
import numpy as np
import re
import unidecode

from tqdm.notebook import tqdm
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

<a id='scraping' />

## Web Scraping
[(back to top)](#content)

In [22]:
#Load webdriver
driver = webdriver.Chrome('/Users/nicolamori/Library/Application Support/binman_chromedriver/mac64/85.0.4183.38/chromedriver')

#Open page
driver.get('https://www.autoscout24.it/lst/fiat/500?lm_premium=true&sort=age&desc=1&offer=U&ustate=U&size=20&page=1&cy=I&fregfrom=2006')
time.sleep(5)

In [23]:
#Detect popup and swtich to it
iframe = driver.find_element_by_xpath('//*[@id="gdpr-consent-notice"]')
driver.switch_to.frame(iframe)

#Close popup
driver.find_element_by_xpath('//*[@class="accept-all mat-button mat-button-base mat-raised-button"]').click()

#Swtich to the original frame
driver.switch_to.default_content()

In [14]:
driver.close()